## Observations and Insights 

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from sklearn import datasets
from scipy.stats import linregress

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# mouse_metadata



In [9]:
# study_results

In [13]:
# Checking the number of mice.
#the .count method will include the duplicate
# mice_number = combine_data_df.count()["Mouse ID"]
# mice_number

#.nunique will not include the duplicate
mice_number = combine_data_df.nunique()["Mouse ID"]
mice_number
                            

NameError: name 'combine_data_df' is not defined

In [11]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
# instead of showing True/False, we use .loc function to show the actual mosue ID
# adding subset so it doesnt check all the duplicate of all values, so now we only checking mosue ID and timepoint
duplicate_mice = combine_data_df.loc[combine_data_df.duplicated(subset = ["Mouse ID", "Timepoint"]), "Mouse ID"].unique()
duplicate_mice



NameError: name 'combine_data_df' is not defined

In [12]:
# Optional: Get all the data for the duplicate mouse ID. 

duplicate_df = combine_data_df.loc[combine_data_df['Mouse ID']== 'g989']
duplicate_df


NameError: name 'combine_data_df' is not defined

In [5]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.

cleaned_df = combine_data_df.loc[combine_data_df['Mouse ID']!= 'g989']
cleaned_df.head()


NameError: name 'combine_data_df' is not defined

In [6]:
# Checking the number of mice in the clean DataFrame.
clean_data_number = cleaned_df.nunique()["Mouse ID"]
clean_data_number
                   



NameError: name 'cleaned_df' is not defined

## Summary Statistics

In [7]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# clean_data_mean = cleaned_df["Tumor Volume (mm3)"].mean()
# clean_data_mean

# mean of the tumor volume for each regimen
mean_groupby_tv = cleaned_df.groupby("Drug Regimen").mean()["Tumor Volume (mm3)"]
# mean_groupby_tv

# median of the tumor volume for each regimen
median_groupby_tv = cleaned_df.groupby("Drug Regimen").median()["Tumor Volume (mm3)"]
# median_groupby_tv

# variance of the tumor volume for each regimen
var_groupby_tv = cleaned_df.groupby("Drug Regimen").var()["Tumor Volume (mm3)"]
# var_groupby_tv

# standard deviation of the tumor volume for each regimen
sd_groupby_tv = cleaned_df.groupby("Drug Regimen").std()["Tumor Volume (mm3)"]
# sd_groupby_tv

# SEM of the tumor volume for each regimen
sem_groupby_tv = cleaned_df.groupby("Drug Regimen").sem()["Tumor Volume (mm3)"]
# sem_groupby_tv



NameError: name 'cleaned_df' is not defined

In [ ]:
# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.


summary_df = pd.DataFrame({
    "Mean Tumor Volume": mean_groupby_tv,
    "Median Tumor Volume": median_groupby_tv,
    "Tumor Volume Variance": var_groupby_tv,
    "Tumor Volume Std. Dev.": sd_groupby_tv,
    "Tumor Volume Std. Err.": sem_groupby_tv
})
summary_df

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line


aggregate_clean_df = cleaned_df.groupby("Drug Regimen").agg({
    "Tumor Volume (mm3)": ['mean', 'median', 'std', 'var', 'sem']
})
aggregate_clean_df

# aggregate-dlfkdfj = clean_df.groupby("drug regimen")
# aggregate-dlfkdfj = aggregate-dlfkdfj.agg("Tumor Volume (mm3)": ['mean', 'median', 'std', 'var', 'sem'])


## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pandas.

# finding the number of data for each drug regimen
regimen_counts = cleaned_df['Drug Regimen'].value_counts()
# regimen_counts

# alternative method of finding the data for each drug regimen 
# regimen_counts = cleaned_df.groupby("Drug Regimen").count()["Mouse ID"]
# regimen_counts

# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pandas.
panda_plot_bar = regimen_counts.plot(kind="bar", figsize=(6,4))

plt.xlabel("Drug Regimen")
plt.ylabel("Number of Unique Mice Tested")





In [ ]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pyplot.


# finding the number of data for each drug regimen
regimen_counts = cleaned_df['Drug Regimen'].value_counts()
# regimen_counts

# Set the x_axis as length of the regimen counts. Need to import numpy as np above 
x_axis = np.arange(len(regimen_counts))
x_axis

# Setting up tick location and use Index.values attribute to return an array representing the data in the given Index object.
# the index.values function will show the drugs regimen name, as opposed to 1,2,3,4 
tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, regimen_counts.index.values)

# creating the bar chart
plt.bar(x_axis, regimen_counts, width = 0.5)

# Rotate the x_axis name to 90 degrees 
plt.xticks(rotation=90)

plt.xlabel("Drug Regimen")
plt.ylabel("Number of Unique Mice Tested")



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas

# perform gender count
gender_counts = cleaned_df["Sex"].value_counts()
# gender_counts


# using panda method to plot
panda_pie_plot = gender_counts.plot(kind="pie", autopct="%1.1f%%")


In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot


# finding how many genders are there
# sex = cleaned_df["Sex"].nunique()


# perform gender count
gender_counts = cleaned_df["Sex"].value_counts()
# gender_counts


# create pie chart 
# adding the .index.values behind the gender_counts to hide the number 
plt.pie(gender_counts, labels=gender_counts.index.values,
        autopct="%1.1f%%")
plt.title('Sex')


# Altnerative fancy way 
# colors = ["lightblue", "orange"]
# explode = (0.1,0)

# plt.pie(gender_counts, explode=explode, labels=gender, colors=colors,
#         autopct="%1.1f%%", startangle=140)

#----------------------------------
# #find the number of female mice and male mice 
# gender_mice = cleaned_df.groupby("Sex").nunique()["Mouse ID"]
# # gender_mice

# # Converting to dataframe, 
# gender_mice_df = gender_mice.to_frame(name="Total Count")
# # gender_mice_df 

# #adding new columns to dataframe and convert to the percentage 
# gender_mice_df["Percentage of Mice"] = (gender_mice_df["Total Count"]/clean_data_number*100).map("{:.1f}%".format)
# gender_mice_df 


## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse

# finding the max timepoint by Mouse ID
max_timepoint_df = pd.DataFrame(cleaned_df.groupby('Mouse ID').max()['Timepoint'].sort_values())
max_timepoint_df 


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint
# merging the max timepoint to the dataframe 
merged_df = pd.merge(cleaned_df, max_timepoint_df, on='Mouse ID')
# merged_df.head(30)

# rename it to max-timepoint to prevent confusion 
renamed_merged_df = merged_df.rename(columns={"Timepoint_y": "Greatest Timepoint", 
                                             "Timepoint_x": "Timepoint"})
renamed_merged_df.head()



In [ ]:
# we need 4 drug regimens - Capomulin, Ramicane, Infubinol, and Ceftamin
# Put treatments into a list for for loop (and later for plot labels)
drugs = ['Capomulin', 'Ramicane', 'Infubinol', 'Ceftamin']


# Create empty list to fill with tumor vol data (for plotting)
tumor_vol_data = []

# Locate each drug from our list of 'drugs' in the orginal dataframe
for drug in drugs:
    drug_df = renamed_merged_df.loc[renamed_merged_df["Drug Regimen"] == drug]
# drug_df.head()

    # only contain final timepoint for each drug regimen 
    final_timepoint_df = drug_df.loc[drug_df['Timepoint'] == drug_df['Greatest Timepoint']]
# final_timepoint_df.head()   

    tumor_values = final_timepoint_df['Tumor Volume (mm3)']
    tumor_vol_data.append(tumor_values)
 
    # Calculate the IQR and quantitatively determine if there are any potential outliers


    quartiles = tumor_values.quantile([.25,.5,.75])
    lowerq = quartiles[0.25]
    upperq = quartiles[0.75]
    iqr = upperq-lowerq
#     print(f'IQR for {drug}: {iqr}')
    
    # Find upper and lower bounds to help identify outliers for each drug regimen
    lower_bound = lowerq - (1.5*iqr)
    upper_bound = upperq + (1.5*iqr)
#     print(f'Lower Bound for {drug}: {lower_bound}')
#     print(f'Upper Bound for {drug}: {upper_bound}')
    
  
    outliers_check = (tumor_values.loc[(final_timepoint_df['Tumor Volume (mm3)'] > upper_bound) | 
                                        (final_timepoint_df['Tumor Volume (mm3)'] < lower_bound)])
    print(f"{drug}'s potential outliers: {outliers_check}")
    

    

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest
# worked with Jessica, got below code together
plt.boxplot(tumor_vol_data)
plt.ylabel('Final Tumor Volume (mm3)')
plt.xticks([1, 2, 3, 4], ['Capomulin', 'Ramicane', 'Infubinol', 'Ceftamin'])

## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin



# finding out what are the mouse that treated with capomulin drug and pick one from the Mouse ID. 
mouse_w_campomulin_df = cleaned_df.loc[cleaned_df['Drug Regimen'] == 'Capomulin']
# mouse_w_campomulin_df

# pick the mouse ID with l509, now we have data of Mouse l509 and is treated by drug Capomulin
mouse_id_w_campomulin = mouse_w_campomulin_df.loc[mouse_w_campomulin_df['Mouse ID'] == 'l509']
# mouse_id_w_campomulin

# plotting the x_axis (timepoint) and y_axis (tumor volume)
plt.plot(mouse_id_w_campomulin['Timepoint'], mouse_id_w_campomulin['Tumor Volume (mm3)'])

# Add labels and title to plot
plt.xlabel("Timepoint (days)")
plt.ylabel("Tumor Volume (mm3)")
plt.title("Capomulin Treatment of Mouse l509")

plt.show()

# Alternative method 
# setting x_axis to be the timepoint of the mouse ID l509
# x_axis = mouse_id_w_campomulin['Timepoint']
# x_axis

# y_axis = mouse_id_w_campomulin['Tumor Volume (mm3)'].map("{:.0f}".format)
# y_axis

# plt.plot(x_axis, y_axis)

# plt.show()

In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen

# showing data of only Capomulin Regimen
campomulin_regimen_df = cleaned_df.loc[cleaned_df['Drug Regimen'] == 'Capomulin']
# campomulin_regimen_df


# calculating the average tumor volume of mouse that was treated by Capomulin regimen
avg_tumor_vol = campomulin_regimen_df.groupby("Mouse ID").mean()["Tumor Volume (mm3)"]
# avg_tumor_vol

# merging average tumor volumes to the dataframe by Mouse ID 
avg_tumor_vol_df = pd.merge(campomulin_regimen_df, avg_tumor_vol, on='Mouse ID')
# avg_tumor_vol_df

# renamed the columns to Average Tumor Volumes to prevent confusinon 
avg_tumor = avg_tumor_vol_df.rename(columns={"Tumor Volume (mm3)_y": "Average Tumor Volume", 
                                             "Tumor Volume (mm3)_x": "Tumor Volume (mm3)"})
# avg_tumor.head(30)




In [ ]:
# we will have to drop out the duplicate 
no_duplicate_df = avg_tumor[['Weight (g)', 'Average Tumor Volume']].drop_duplicates()
no_duplicate_df.head()

# the 0, 10, 18 means when whenever it reaches 10, 18, it is a different Mouse ID number. 
# We are only taking the data of the first mice to present. Not the mice from all timepoint


In [ ]:
# Setting up the X and Y axis 
x_axis = no_duplicate_df['Weight (g)']
y_axis = no_duplicate_df['Average Tumor Volume']

# making a scatter plot
plt.scatter(x_axis, y_axis)

# giving the name to the x and y label
plt.xlabel("Weight (g)")
plt.ylabel("Average Tumor Volume (mm3)")



## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 

# for mouse weight and average tumor volume for the Capomulin regimen
x_axis = no_duplicate_df['Weight (g)']
y_axis = no_duplicate_df['Average Tumor Volume']

# giving the name to the x and y label
plt.xlabel("Weight (g)")
plt.ylabel("Average Tumor Volume (mm3)")

# ----------------------------above code are from previous scatter plot 

# finding the correlation 
correlation = st.pearsonr(x_axis, y_axis)
correlation
print(f"The correlation between mouse weight and the average tumor volume is {round(correlation[0],2)}")
#the correlation coefficient between weight(g)and average tumor volume is 0.84


# go back up and add this -> scipy.stats import linregress
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
regress_values = x_axis * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_axis, y_axis)
plt.plot(x_axis,regress_values,"r-")
# the code below is to show the line equation
# plt.annotate(line_eq,(20,40),fontsize=10,color="red")
plt.show()